In [188]:
import numpy as np
import xlrd
import pandas as pd
import csv
import sqlite3
import matplotlib
#matplotlib.use("svg")
import matplotlib.pyplot as plt
import os
if not os.environ.get("DISABLE_TESTING", False):
    %matplotlib inline
plt.style.use('ggplot')
import re
from sklearn import linear_model
import statsmodels.api as sm

UsageError: Invalid GUI request 'svg', valid ones are:dict_keys(['inline', 'nbagg', 'notebook', 'ipympl', 'widget', None, 'qt4', 'qt', 'qt5', 'wx', 'tk', 'gtk', 'gtk3', 'osx', 'asyncio'])


In [189]:
#Load graduation_rate_total 
conn = sqlite3.connect(":memory:")
cursor = conn.cursor()

cursor.execute("""
CREATE TABLE GRAD_RATE_TOTAL(
    region_id INTEGER,
    region_name TEXT,
    region_type TEXT,
    year INTEGER,
    variable TEXT,
    percentage REAL  
);""")

conn.commit()


with open ("graduation_rates_total.csv",encoding='latin-1') as grad_rate:
    data = csv.reader(grad_rate)
    counter = 0
    for row in data:
        if counter >= 1:
            cursor.execute("INSERT INTO GRAD_RATE_TOTAL VALUES (?,?,?,?,?,?)",tuple(row))
        counter += 1
        
conn.commit()



#pd.read_sql_query("SELECT * from GRAD_RATE_TOTAL;", conn)

In [190]:
fin_df = pd.read_excel("financial_data.xls", sheet_name=["14","15","16","17","18"], skiprow=5)
fin_df_pps = fin_df["18"].fillna(-1)

In [191]:
# print(fin_df["18"].columns)
per_pupil_spending = pd.DataFrame([], columns=["Rank", "School System", "State", "Enrollment", "Instruction_Total", 
                              "Instruction_Salaries", "Instruction_EmployeeBenefits", "Support_Total", 
                              "Support_Pupil", "Support_Staff","Support_GeneralAdmin", "Support_SchoolAdmin"])
per_pupil_spending["Rank"] = [i for i in range(100)]
per_pupil_spending["School System"] = [i for i in fin_df_pps["Unnamed: 2"][4:] if i !="School System" and i != -1]
per_pupil_spending["State"] = [i for i in fin_df_pps["Unnamed: 3"][4:] if i !="State" and i != -1]
per_pupil_spending["Enrollment"] = [int(i) for i in fin_df_pps["Unnamed: 4"] if i != -1 and type(i) != str]
per_pupil_spending["Instruction_Total"] = [int(i) for i in fin_df_pps["Unnamed: 8"] if i != -1 and type(i) != str]
per_pupil_spending["Instruction_Salaries"] = [int(i) for i in fin_df_pps["Unnamed: 9"] if i != -1 and type(i) != str]
per_pupil_spending["Instruction_EmployeeBenefits"] = [int(i) for i in fin_df_pps["Unnamed: 10"] if i != -1 and type(i) != str]
per_pupil_spending["Support_Total"] = [int(i) for i in fin_df_pps["Unnamed: 11"] if i != -1 and type(i) != str]
per_pupil_spending["Support_Pupil"] = [int(i) for i in fin_df_pps["Unnamed: 12"] if i != -1 and type(i) != str]
per_pupil_spending["Support_Staff"] = [int(i) for i in fin_df_pps["Unnamed: 13"] if i != -1 and type(i) != str]
per_pupil_spending["Support_GeneralAdmin"] = [int(i) for i in fin_df_pps["Unnamed: 14"] if i != -1 and type(i) != str]
per_pupil_spending["Support_SchoolAdmin"] = [int(i) for i in fin_df_pps["Unnamed: 15"] if i != -1 and type(i) != str]

#per_pupil_spending.to_csv(index=False)

In [192]:
#Finance sheet 17
fin_df_percent_dist_rev = fin_df["17"].fillna(-1)

In [193]:
percent_dist_rev = pd.DataFrame([], columns=["Rank","School System","State","Enrollment","Overall Total Percentage",
                                             "Federal Total Percentage","Federal Title I","State Total Percentage",
                                             "State General Formula Assistance","Local Total Percentage",
                                             "Local Taxes Parent Government Contributions","Other Local Government",
                                            "Charges"])

percent_dist_rev["Rank"] = [i for i in range(100)]
percent_dist_rev["School System"] = [i for i in fin_df_percent_dist_rev["Unnamed: 2"][4:] if i !="School System" and i != -1]
percent_dist_rev["State"] = [i for i in fin_df_percent_dist_rev["Unnamed: 3"][4:] if i !="State" and i != -1]
percent_dist_rev["Enrollment"] = [int(i) for i in fin_df_percent_dist_rev["Unnamed: 4"] if i != -1 and type(i) != str]
percent_dist_rev["Overall Total Percentage"] = [float(i) for i in fin_df_percent_dist_rev["Unnamed: 5"][4:] if i !="Total" and i != -1]
percent_dist_rev["Federal Total Percentage"] = [float(i) for i in (fin_df_percent_dist_rev["Unnamed: 6"]) if i !="Federal sources" and i != "Total2" and i != -1]
percent_dist_rev["Federal Title I"] = [float(i) for i in (fin_df_percent_dist_rev["Unnamed: 7"]) if i !="Federal sources" and i != "Title I" and i != -1]
percent_dist_rev["State Total Percentage"] = [float(i) for i in (fin_df_percent_dist_rev["Unnamed: 8"]) if i !="State sources" and i != "Total2" and i != -1]
percent_dist_rev["State General Formula Assistance"] = [float(i) for i in (fin_df_percent_dist_rev["Unnamed: 9"]) if i !="State sources" and i != "General" and i !="formula" and i != "assistance" and i != -1]
percent_dist_rev["Local Total Percentage"] = [float(i) for i in (fin_df_percent_dist_rev["Unnamed: 10"]) if i !="Local sources" and i != "Total2" and i != -1]

#This function changes the (Z) to zeros  
def changeZ(i):
    try:
        return (float(i))
    except:
        return(0.0)

percent_dist_rev["Local Taxes Parent Government Contributions"] = [changeZ(i) for i in (fin_df_percent_dist_rev["Unnamed: 11"]) if i !="Local sources" and i != -1 and i!="Taxes and" and i != "parent" and i != "government" and i != "contributions"]
percent_dist_rev["Other Local Government"] = [changeZ(i) for i in (fin_df_percent_dist_rev["Unnamed: 12"]) if i !="Local sources" and i != -1 and i!="Other" and i != "local" and i != "governments"]
percent_dist_rev["Charges"] = [changeZ(i) for i in (fin_df_percent_dist_rev["Unnamed: 13"]) if i !="Local sources" and i != -1 and i!="Charges"]


#percent_dist_rev.to_csv(index=False)

In [194]:
#Finance sheet 16
fin_df_expenditure = fin_df["16"].fillna(-1)

In [195]:
#Finance Sheet 16
expenditure = pd.DataFrame([], columns=["Rank","School System","State","Enrollment",
                                       "Elementary_secondary Expenditure Total", "Selected Objects Salaries",
                                       "Selected Objects Employee Benfits","Instruction Salaries","Instruction Employee Benefits",
                                       "Support Services Pupil Support","Support Services Instructive Staff Support",
                                       "Support Services General Administration","Support Services School Administration",
                                       "Support Services Other and Nonspecified","Other Current Spending","Capital Outlay",
                                       "Inter-governmental","Interest On Debt"])

expenditure["Rank"] = [i for i in range(100)]
expenditure["School System"] = percent_dist_rev["School System"]
expenditure["State"] = percent_dist_rev["State"]
expenditure["Enrollment"] = percent_dist_rev["Enrollment"]
expenditure["Elementary_secondary Expenditure Total"] = [int(i) for i in fin_df_expenditure["Unnamed: 5"][np.concatenate((np.arange(4,74,1),np.arange(149,217,1)))] if i !="Total" and i != -1 and i != "Elementary-secondary expenditure"]
expenditure["Selected Objects Salaries"] = [int(i) for i in fin_df_expenditure["Unnamed: 7"][np.concatenate((np.arange(4,74,1),np.arange(149,217,1)))] if i !="For selected objects" and i != -1 and i != "Salaries and" and i != "wages"]
expenditure["Selected Objects Employee Benfits"] = [int(i) for i in fin_df_expenditure["Unnamed: 8"][np.concatenate((np.arange(4,74,1),np.arange(149,217,1)))] if i != -1 and i != "Employee" and i != "benefits"]
expenditure["Instruction Salaries"] = [int(i) for i in fin_df_expenditure["Unnamed: 10"][np.concatenate((np.arange(4,74,1),np.arange(149,217,1)))] if i != -1 and i != "Salaries and" and i != "wages"]
expenditure["Instruction Employee Benefits"] = [int(i) for i in fin_df_expenditure["Unnamed: 11"][np.concatenate((np.arange(4,74,1),np.arange(149,217,1)))] if i != -1 and i != "Employee" and i != "benefits"]
expenditure["Support Services Pupil Support"] = [int(i) for i in fin_df_expenditure["Unnamed: 5"][np.concatenate((np.arange(76,145,1),np.arange(220,288,1)))] if i != -1 and i != "Pupil" and i != "support"]
expenditure["Support Services Instructive Staff Support"] = [int(i) for i in fin_df_expenditure["Unnamed: 6"][np.concatenate((np.arange(76,145,1),np.arange(220,288,1)))] if i != -1 and i != "Instructional" and i != "staff" and i != "support"]
expenditure["Support Services General Administration"] = [int(i) for i in fin_df_expenditure["Unnamed: 7"][np.concatenate((np.arange(76,145,1),np.arange(220,288,1)))] if i != -1 and i != "General" and i != "adminis-" and i != "tration"]
expenditure["Support Services School Administration"] = [int(i) for i in fin_df_expenditure["Unnamed: 8"][np.concatenate((np.arange(76,145,1),np.arange(220,288,1)))] if i != -1 and i != "School" and i != "adminis-" and i != "tration"]
expenditure["Support Services Other and Nonspecified"] = [int(i) for i in fin_df_expenditure["Unnamed: 9"][np.concatenate((np.arange(76,145,1),np.arange(220,288,1)))] if i != -1 and i != "Other and" and i != "nonspecified"]
expenditure["Other Current Spending"] = [int(i) for i in fin_df_expenditure["Unnamed: 10"][np.concatenate((np.arange(76,145,1),np.arange(220,288,1)))] if i != -1 and i != "Other" and i != "current" and i != "spending"]
expenditure["Capital Outlay"] = [int(i) for i in fin_df_expenditure["Unnamed: 11"][np.concatenate((np.arange(76,145,1),np.arange(220,288,1)))] if i != -1 and i != "Capital" and i != "outlay"]
expenditure["Inter-governmental"] = [int(i) for i in fin_df_expenditure["Unnamed: 12"][np.concatenate((np.arange(76,145,1),np.arange(220,288,1)))] if i != -1 and i != "Inter-" and i != "governmental"]
expenditure["Interest On Debt"] = [int(i) for i in fin_df_expenditure["Unnamed: 13"][np.concatenate((np.arange(76,145,1),np.arange(220,288,1)))] if i != -1 and i != "Interest" and i != "on debt"]


#expenditure.to_csv(index=False, )

In [196]:
#load grad_rate_total.csv using pandas
grad_rate = pd.read_csv("graduation_rates_total.csv",encoding='latin-1')

In [197]:
#only take the grad rate of the most recent year 
#generate a new data frame
new_frame = []
grad_rate_sort_year = grad_rate.sort_values(by='year',ascending=False)
grad_rate_sort_year_sorted = grad_rate_sort_year.groupby(["name"])
for g in grad_rate_sort_year_sorted:
    new_frame.append(g[1].iloc[0,:])


reduced_grad_rate = pd.DataFrame(new_frame)
#new grad rate data frame with the most recent grad rate
reduced_grad_rate = reduced_grad_rate.reset_index().drop(["index"],axis=1)


#collect the counties and states separately in the grad rate data 
#for matching purposes
county = []
state = []

for name in reduced_grad_rate["name"]:
    county_name, state_name = name.split(", ")
    county.append(county_name)
    state.append(state_name)
    
#change all the county name to ones without the trailing "county"
tmp = []
for i in range(len(county)):
        c = county[i]
        k = [j for j in c.split(" County") if j != ""]
        tmp = tmp+k
        
county = tmp

In [198]:
school_system = per_pupil_spending["School System"]

#list of indices for the per_pupil_spending data frame 
#names can be found in grad rate data
school_final_ind = []

#indices of the grad rate data that match up the per pupil spending
grad_rate_ind = []

#graduation rate in order of the per pupil spending data frame
grad_rate_values = []

for i in range(len(per_pupil_spending)):
    school = school_system[i]
    for j in range(len(county)):
        c = county[j]
        s = state[j]
        if (c.lower().replace("'","") in school.lower().replace("'","") and s==per_pupil_spending["State"][i]):
            school_final_ind.append(i)
            grad_rate_ind.append(j)
            grad_rate_values.append(reduced_grad_rate["value"][j])
            
per_pupil_spending_with_grad = per_pupil_spending.loc[school_final_ind,:]
per_pupil_spending_with_grad["grad_rate"] = grad_rate_values 

per_pupil_spending_with_grad.to_csv("per_pupil_spending_with_grad.csv",index=False)

In [199]:
#Remove outliers and then plot


def outliers_rem(features,df):
    features_dict = {}
    for feature in features:
        description = df[feature].describe()
        feature_mean = description["mean"]
        feature_std_3 = description["std"]*3
        features_dict[feature] = (feature_mean + feature_std_3,feature_mean - feature_std_3)
    return features_dict

def EDA(features_dict, df,log = False):
    for feature in features_dict:
        tmp = df[df[feature]<=features_dict[feature][0]]
        tmp = tmp[tmp[feature]>=features_dict[feature][1]]
        if log:
            plt.scatter(np.log(tmp[feature]),np.log(tmp["grad_rate"]),color="red")
            plt.xlabel(feature + " in log scale")
        else:
            plt.scatter(tmp[feature],tmp["grad_rate"],color="red")
            plt.xlabel(feature)
        plt.ylabel("Graduation Rate")
        plt.title(feature+" vs. Graduation Rate")
        plt.show()
        
        plt.hist(tmp[feature])
        plt.title("Histogram of "+feature)
        plt.show()
    
    
    

features = ["Instruction_EmployeeBenefits","Support_Pupil","Support_Staff","Support_SchoolAdmin"]

a = outliers_rem(features,per_pupil_spending_with_grad)

EDA(a,per_pupil_spending_with_grad)

#features_dict = {}

#for feature in features:
    #description = per_pupil_spending_with_grad[feature].describe()
    #feature_mean = description["mean"]
    #feature_std_3 = description["std"]*3
    #features_dict[feature] = feature_mean + feature_std_3


#for feature in features_dict:
    #tmp = per_pupil_spending_with_grad[per_pupil_spending_with_grad[feature]<=features_dict[feature]]
    #plt.scatter(np.log(tmp[feature]),tmp["grad_rate"],color="blue")
    #plt.xlabel(feature)
    #plt.ylabel("Graduation Rate")
    #plt.title(feature+" vs. Graduation Rate")
    #plt.show()
    #plt.hist(tmp[feature])
    #plt.title("Histogram of "+feature)
    #plt.show()
    
    
    
    

NameError: name 'plt' is not defined

In [203]:
reg = linear_model.LinearRegression()
per_pupil_spending_with_grad
#reg.fit(per_pupil_spending_with_grad[["Instruction_EmployeeBenefits","Support_Pupil","Support_Staff","Support_SchoolAdmin"]],per_pupil_spending_with_grad["grad_rate"])
#reg.coef_
results = sm.OLS(per_pupil_spending_with_grad["grad_rate"], per_pupil_spending_with_grad[["Instruction_EmployeeBenefits","Support_Pupil","Support_Staff","Support_SchoolAdmin"]]).fit()
print(results.summary())

NameError: name 'linear_model' is not defined

In [204]:
fin_df_tot_rev = fin_df["15"].fillna(-1)
#Revenue of the 100 Largest .....
#Finance page 15
tot_rev = pd.DataFrame([], columns=["Rank","School System","State","Enrollment",
                                   "Total Revenue","Federal Total","Federal Titile I","Federal Special Education",
                                   "Federal Child Nutrition","Federal Other and Nonspecified","State Total","State General Formula Assistance",
                                   "State Special Education","State Transportation","State Other and Nonspecified","Local Total",
                                   "Local Total Taxes","Local Property Taxes","Local Parent Government Contribution","Local From Cities and Counties",
                                   "Local From Other School Systems"])

def changeInt(i):
    try:
        return (int(i))
    except:
        return(0)

tot_rev["Rank"] = [i for i in range(100)]
tot_rev["School System"] = percent_dist_rev["School System"]
tot_rev["State"] = percent_dist_rev["State"]
tot_rev["Enrollment"] = percent_dist_rev["Enrollment"]
tot_rev["Total Revenue"] = [int(i) for i in fin_df_tot_rev["Unnamed: 5"][np.concatenate((np.arange(4,74,1),np.arange(149,217,1)))] if i !="Total" and i != -1 and i != "Elementary-secondary revenue"]
tot_rev["Federal Total"] = [int(i) for i in fin_df_tot_rev["Unnamed: 6"][np.concatenate((np.arange(4,74,1),np.arange(149,217,1)))] if i !="Total" and i != -1 and i != "Federal sources"]
tot_rev["Federal Titile I"] = [int(i) for i in fin_df_tot_rev["Unnamed: 7"][np.concatenate((np.arange(4,74,1),np.arange(149,217,1)))] if i !="Title I" and i != -1 ]
tot_rev["Federal Special Education"] = [changeInt(i) for i in fin_df_tot_rev["Unnamed: 8"][np.concatenate((np.arange(4,74,1),np.arange(149,217,1)))] if i !="Special" and i != -1 and i != "education"]/tot_rev["Enrollment"]
tot_rev["Federal Child Nutrition"] = [changeInt(i) for i in fin_df_tot_rev["Unnamed: 9"][np.concatenate((np.arange(4,74,1),np.arange(149,217,1)))] if i !="Child" and i != -1 and i != "nutrition"]/tot_rev["Enrollment"]
tot_rev["Federal Other and Nonspecified"] = [changeInt(i) for i in fin_df_tot_rev["Unnamed: 10"][np.concatenate((np.arange(4,74,1),np.arange(149,217,1)))] if i !="Child" and i != -1 and i != "Other and" and i != "nonspecified"]
tot_rev["State Total"] = [changeInt(i) for i in fin_df_tot_rev["Unnamed: 11"][np.concatenate((np.arange(4,74,1),np.arange(149,217,1)))] if i !="State sources" and i != -1 and i != "Total"]
tot_rev["State General Formula Assistance"] = [changeInt(i) for i in fin_df_tot_rev["Unnamed: 12"][np.concatenate((np.arange(4,74,1),np.arange(149,217,1)))] if i !="General" and i != -1 and i != "formula" and i != "assistance"]
tot_rev["State Special Education"] = [changeInt(i) for i in fin_df_tot_rev["Unnamed: 13"][np.concatenate((np.arange(4,74,1),np.arange(149,217,1)))] if i !="Special" and i != -1 and i != "education"]/tot_rev["Enrollment"]
tot_rev["State Transportation"] = [changeInt(i) for i in fin_df_tot_rev["Unnamed: 4"][np.concatenate((np.arange(76,145,1),np.arange(220,288,1)))] if i !="State sources--Con." and i != -1 and i != "Transpor-" and i != "tation" and i !="Elementary-secondary revenue--Con."]/tot_rev["Enrollment"]
tot_rev["State Other and Nonspecified"] = [changeInt(i) for i in fin_df_tot_rev["Unnamed: 5"][np.concatenate((np.arange(76,145,1),np.arange(220,288,1)))] if i !="Other and" and i != -1 and i != "nonspecified"]
tot_rev["Local Total"] = [changeInt(i) for i in fin_df_tot_rev["Unnamed: 6"][np.concatenate((np.arange(76,145,1),np.arange(220,288,1)))] if i !="Local sources" and i != -1 and i != "Total"]
tot_rev["Local Total Taxes"] = [changeInt(i) for i in fin_df_tot_rev["Unnamed: 7"][np.concatenate((np.arange(76,145,1),np.arange(220,288,1)))] if i !="Total taxes" and i != -1]
tot_rev["Local Property Taxes"] = [changeInt(i) for i in fin_df_tot_rev["Unnamed: 8"][np.concatenate((np.arange(76,145,1),np.arange(220,288,1)))] if i !="Property" and i != -1 and i != "taxes"]/tot_rev["Enrollment"]
tot_rev["Local Parent Government Contribution"] = [changeInt(i) for i in fin_df_tot_rev["Unnamed: 9"][np.concatenate((np.arange(76,145,1),np.arange(220,288,1)))] if i !="Parent" and i != -1 and i != "government" and i != "contributions"]
tot_rev["Local From Cities and Counties"] = [changeInt(i) for i in fin_df_tot_rev["Unnamed: 10"][np.concatenate((np.arange(76,145,1),np.arange(220,288,1)))] if i !="From cities" and i != -1 and i != "and" and i != "counties"]
tot_rev["Local From Other School Systems"] = [changeInt(i) for i in fin_df_tot_rev["Unnamed: 11"][np.concatenate((np.arange(76,145,1),np.arange(220,288,1)))] if i !="From other" and i != -1 and i != "systems" and i != "school"]

tot_rev_with_grad = tot_rev.loc[school_final_ind,:]
tot_rev_with_grad["grad_rate"] = grad_rate_values

tot_rev.columns

tot_rev_with_grad.to_csv("tot_rev_with_grad.csv",index=False)

In [ ]:
features_rev = ['Federal Child Nutrition','Federal Special Education','State Special Education',
               'State Transportation','Local Property Taxes']

b = outliers_rem(features_rev, tot_rev_with_grad)

EDA(b,tot_rev_with_grad)

In [ ]:
# Grad Rates by Race

grad= pd.read_csv("grad_rates_by_race.csv", sep="\t", lineterminator="\r")

In [207]:
import re

selected_cols = ['State']
for i in range(10,len(grad.columns)):
#     print(grad[grad.columns[i]].isna().sum())
    if grad[grad.columns[i]].isna().sum() != 77: selected_cols += [grad.columns[i]]
# grad_parsed = grad[selected_cols]

selected_rows = []
for i in range(len(grad)-13):
#     print(grad[i:i+1].isna().sum().sum())
    if (grad[i:i+1].isna().sum().sum() != 25): selected_rows += [i]

grad_parsed = grad[selected_cols].iloc[selected_rows].reset_index()
grad_parsed = grad_parsed.drop('index', axis=1).drop([3], axis=0)

header = ["State","Total", "White", "Black", "Hispanic", "Asian/Pacific Islander", "Asian", "Pacific Islander", 
          "American Indian/Alaska/Native", "Two or more races", "Students with Disabilities", 
          "Limited English", "Economically disadvantaged"]

grad_parsed.columns = header
grad_parsed = grad_parsed[3:].reset_index().drop('index', axis=1)

states=[]
for i in grad_parsed['State']:
    states += [re.sub("[^a-zA-Z]+", "", i)]
grad_parsed["State"] = states

grad_parsed.to_csv('grad_rates_by_race_parsed.csv', index=False)

In [208]:
agcr = pd.read_csv('grad_rates_by_race_parsed.csv')

In [206]:
means_by_dem = dict()
for col in range(2,len(agcr.columns)):
    ok = []
    for row in range(len(agcr)):
        if type(agcr.iloc[row,col]) == np.int64 or agcr.iloc[row,col].isdigit(): 
            ok += [int(agcr.iloc[row,col]) - agcr.iloc[row,1]]
    means_by_dem[agcr.columns[col]] = np.mean(ok)

# Average difference in percentage between racial demographic and total
means_by_dem

#plt.bar(means_by_dem.keys(), means_by_dem.values(), color='g')
#plt.xticks(rotation=90)

{'White': 3.3653846153846154,
 'Black': -7.403846153846154,
 'Hispanic': -6.134615384615385,
 'Asian/Pacific Islander': 4.865384615384615,
 'Asian': 5.487179487179487,
 'Pacific Islander': -4.580645161290323,
 'American Indian/Alaska/Native': -10.0,
 'Two or more races': -2.3488372093023258,
 'Students with Disabilities': -17.705882352941178,
 'Limited English': -19.68,
 'Economically disadvantaged': -7.392156862745098}

In [246]:
national_math_1 = pd.read_csv("National_math_2017.csv",encoding='latin-1', na_values = 0)
national_reading_1 = pd.read_csv("National_reading_2017.csv",encoding='latin-1', na_values = 0)
national_science_1 = pd.read_csv("National_science_2017.csv",encoding='latin-1', na_values = 0)
national_writing_1 = pd.read_csv("National_writing_2017.csv",encoding='latin-1', na_values = 0) 

In [247]:
national_math = national_math_1.rename({'MN':"Math"}, axis='columns')
national_reading = national_reading_1.rename({'MN':"Reading"}, axis='columns')
national_science = national_science_1.rename({'MN':"Science"}, axis='columns')
national_writing = national_writing_1.rename({'MN':"Writing"}, axis='columns')

In [249]:
national = national_math.merge(national_reading,on='Jurisdiction').merge(national_science,on='Jurisdiction').merge(national_writing,on='Jurisdiction')
national = national[['Jurisdiction', 'Math', 'Science', 'Writing', 'Reading']]





print(type(national))
standard = national.loc[national['Jurisdiction'] == 'National public']

national['Jurisdiction'] = national['Jurisdiction'].str.replace(" ","")
print(national)
std_math = list(dict(standard['Math']).values())[0]
std_sci = list(dict(standard['Science']).values())[0]
std_wri = list(dict(standard['Writing']).values())[0]
std_read = list(dict(standard['Reading']).values())[0]




values = {'Math': std_math, 'Science': std_sci, 'Writing': std_wri, 
          'Reading': std_read}

cash = national.fillna(value=values)
cash['average'] = cash.mean(numeric_only=True, axis=1)


agcr = agcr.rename({'State': 'Jurisdiction'}, axis='columns')
agcr_cond = agcr[['Jurisdiction', 'Total']]
agcr_cond = agcr_cond.rename({'Total':"Grad Rate"}, axis='columns')
cash = cash.merge(agcr_cond,on= "Jurisdiction")

from sklearn.datasets import load_digits
from sklearn.feature_selection import SelectKBest, f_classif

print(cash)
X_new = SelectKBest(f_classif, k=1).fit_transform(cash.iloc[:,1:-1], cash.iloc[:,-1])
pd.DataFrame(X_new)




    
    

<class 'pandas.core.frame.DataFrame'>
          Jurisdiction  Math  Science  Writing  Reading
0        Massachusetts   297    162.0    167.0    278.0
1            Minnesota   294    162.0    156.0    269.0
2         NewHampshire   293    165.0    160.0    275.0
3                DoDEA   293    166.0    165.0    280.0
4            NewJersey   292    156.0    175.0    275.0
5             Virginia   290    160.0    157.0    268.0
6           Washington   289    157.0    158.0    272.0
7              Wyoming   289    160.0    158.0    269.0
8             Nebraska   288    160.0      NaN    269.0
9          NorthDakota   288    161.0    154.0    265.0
10           Wisconsin   288    159.0    158.0    269.0
11                Ohio   288    157.0    156.0    268.0
12             Indiana   288    156.0    155.0    272.0
13             Vermont   288    163.0    162.0    273.0
14                Utah   287    166.0    152.0    269.0
15        Pennsylvania   286      NaN    159.0    270.0
16        

,0
0,278.0
1,269.0
2,275.0
3,275.0
4,268.0
5,272.0
6,269.0
7,269.0
8,265.0
9,269.0
